Implement python method to convert E-field values to photon number density


_________________
Imports

In [16]:
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse

import scipy.constants as const

import srwlib
from srwlib import srwl

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

_______________
Laser defaults

In [17]:
# Specify parameters
_LASER_PULSE_SLICE_DEFAULTS = PKDict(
    sigx_waist=10.0e-6,  
    sigy_waist=10.0e-6, 
)
_LASER_PULSE_DEFAULTS = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        pulseE=0.001,
        w0=128.9e-6,
        a0=0.01,
        tau_fwhm=0.3e-13,
        slice_params=_LASER_PULSE_SLICE_DEFAULTS,
)

# Instantiate the default parameters:
params = _LASER_PULSE_DEFAULTS.copy()

___________
In this section, can change each parameter one at a time:

photon_e_ev: wavelength 800nm --> 900nm  
pulseE: 0.001 --> 0.002  
w0 = 128.9e-6 --> 200.0e-6  
a0 = 0.01 --> 0.02  
tau_fwhm = 0.3e-13 --> 0.6e-13   

and

sigx_waist = 10.0e-6 --> 20.0e-6  
sigy_waist = 10.0e-6 --> 20.0e-6  

In [18]:

# Calculate phE starting value from wavelength
# lambda0 = 9.0e-7                    # wavelength [m]
# photon_e = const.h * const.c / lambda0   # photon energy [J]
# photon_e_ev = photon_e / const.e              # photon energy [eV]
# params.photon_e_ev = photon_e_ev
# print(' photon energy = {0:4.2E} [J]'.format(photon_e) + ' = {0:4.2f} [eV]'.format(photon_e_ev))


# # pulseE
# pulseE = 0.002
# params.pulseE = pulseE


# # w0
# w0 = 200.0e-6
# params.w0 = w0


# # a0
# a0 = 0.02
# params.a0 = a0


# # tau_fwhm
# tau_fwhm = 0.6e-13                  # FWHM pulse length [s]
# params.tau_fwhm = tau_fwhm


# # sigx_waist and sigy_waist
# sigx_waist = 20.0e-6
# sigy_waist = 20.0e-6
# params.slice_params.sigx_waist = sigx_waist
# params.slice_params.sigy_waist = sigy_waist


Instantiate the laser pulse

In [19]:
# Instantiate the laser pulse
thisPulse = pulse.LaserPulse(params)

# initial pulse - intensity

# choose one of the laser pulse slices, and grab its SRW wavefront object
slice_array=thisPulse.slice
slice_number = 0
wfr0=slice_array[slice_number].wfr

intensity0 = srwlib.array('f', [0]*wfr0.mesh.nx*wfr0.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(intensity0, wfr0, 0, 0, 3, wfr0.mesh.eStart, 0, 0) #extracts intensity

##Reshaping electric field data from flat to 2D array
intens_2d_0 = np.array(intensity0).reshape((wfr0.mesh.nx, wfr0.mesh.ny), order='C')

______________

Obtain E [V/m] from the laser pulse instantiation

In [20]:
# SRW intensity variable: intens_2d_0
# For vacuum: |E|^2 = sqrt(\mu0/\eps0)*2*I

efield_abs_sqrd_2d = np.sqrt(const.mu_0 / const.epsilon_0) * 2.0 * intens_2d_0 # [V^2/m^2]

_______________

Calculate the number of photons in each grid cell of the laser slice (using the wavelength, grid cell size, and laser slice length)

In [21]:
# Calculate the length of the grid cells
dx = (wfr0.mesh.xFin - wfr0.mesh.xStart)/wfr0.mesh.nx
dy = (wfr0.mesh.yFin - wfr0.mesh.yStart)/wfr0.mesh.ny

# Field energy per grid cell is the area of that cell times the energy density
cell_volume = thisPulse.slice[slice_number].ds * dx * dy
energy_2d = cell_volume * (const.epsilon_0 / 2.0) * efield_abs_sqrd_2d

# Get slice value of phE (will be in eV)
photon_e = thisPulse.slice[slice_number].photon_e_ev * const.e

# Number of photons in each grid cell can be found by dividing the total energy of the laser 
# in that grid cell by the energy of a photon (phE is photon energy [J], but params.phE is in [eV]...)
n_photons_2d = energy_2d / photon_e

total_n_photons = np.sum(np.sum(n_photons_2d))
print('Total number of photons in this slice: {0:4.2E}'.format(total_n_photons))

tnp_class_fn = thisPulse.slice[slice_number].calc_init_n_photons()
print('Same number using the class function: {0:4.2E}'.format(np.sum(np.sum(tnp_class_fn))))

Total number of photons in this slice: 1.34E+15
Same number using the class function: 1.34E+15


____________

Check that pulseE and the calculated total energy for a laser with one slice are the same


In [22]:
# User-specified value of pulseE [J]
print('User-specified pulse energy: {0:4.2E} [J]'.format(thisPulse.slice[slice_number].pulseE_slice))
print('Calculated total pulse energy: {0:4.2E} [J]'.format(np.sum(np.sum(energy_2d))))

User-specified pulse energy: 3.33E-04 [J]
Calculated total pulse energy: 3.33E-04 [J]
